# Work Diversity Map

## 1. Define Styling
Description

Example: https://mkz.smartuse.ch/uploads/assets/views/SU-003-Arbeitsplatzdiversitaet.html

**Columns:**  
 - *Size:* Vollzeitäquivalente Total from STATENT (B08VZAT/BVZAT)
 - *Color:* 7 cividi_categories mapped from 96 statent_categories: 
   - Gesundheit & Soziale Einrichtungen
   - Bildung & Forschung
   - Freizeit, Gastronomie, Hotelerie
   - Publikumsorientierte Nutzungen
   - Finanzdienstleistungen
   - Industrie & produzierendes Gewerbe
   - Sonstige Dienstleistungen
 - *Transparency:* diversity of cividi_categories 

## 2.Cookbook
### **Set up envirement**
The [cividi hub Docker](https://github.com/cividi/hub) includes the essential packages: sqlite3 with spatialite, python with geopandas, and datapackage-pipelines with dataflows

The following commands should be run inside the container.
### **Spatialite Querying**

INPUT: requested data through spatial querying  
CODE: *(see below)*  
OUTPUT: <result\>.csv  
``` bash
>>> sqlite3 BFS.db < spatialite_querying.sql
```

To change the year of interest or the region of interest, modify the last line of the file. spatialite_querying.sql
``` sqlite
...
...
WHERE (A.yr=='2015' AND Within( A.geometryLV95,Buffer( GeomFromText( 'Point(2683268 1248001)' ), 5000 ) ) );
```


### **Datapackage pipelines**

INPUT: 'input/result.csv (from Spatialite Querying) + data_to_feed.yaml  
CODE: *(see below)*   
OUTPUT: output/ZHR/datapackage.json, output/ZHR/modified_result.csv  
  
A short description of the pipeline actions: 
 - run: load;
 - flow: flows.remap_categorys
 - flow: flows.calc_log_density
 - flow: flows.calc_diversity
 - flow: flows.styling
 - flow: flows.clean_output
 - run: update_package
 
 
 ### Description of actions:
 
 1. **run: load**
 
 Load data from a csv-file we generated at **Spatialite-Querying**
 2. flow: flows.remap_categorys
 
 The BFS work data is split into 96 Categories witch will be remapped to 7 categories during this flow. Likewise, the biggest new category will be calculated and stored as a column.
 3. flow: flows.calc_log_density
 
 The size of the circle on the map represents the total working density. For a more intuitive visualization, the logarithm of this density is calculated.
 4. flow: flows.calc_diversity

 To calc a score for diversity of categories of work we use the popular diversity index in the ecological literature: Shannon-Index.
 5. flow: flows.styling
 
 Is reading the ```default_dict.json``` to set depending on the category a style for each circle. Afterward, the radius and transparency will be set here, depending on the precalculated data.
 6. flow: flows.clean_output
 
 This flow reduces the data to the necesserly ones for the visualizen:
 ``` 
 'wkt', 'fill', 'fillColor', 'fillOpacity', 'stroke', 'color', 'opacity', 'weight', 'radius'
  ```
  Example: ZH, Hbf + 20km = 23760 rows
   - cleaned output: 
    - size(snapshot.json) = 7 MiB
    - runtime(dpp run ./work_diversity && python exporter.py) = 50s
   - uncleaned output: 
    - size(snapshot.json) = 53 MiB 
    - runtime(dpp run ./work_diversity && python exporter.py) = 150s
            
 7. run: update_package
 This function is adding the required data and information for the snapshot to the data package. Here you can modify the map and its legend.


### **Exporter**

INPUT: datapackage.json, modified_result.csv  
CODE: *(see below)*   
OUTPUT: snapshot.json    

CODE:
```python
import yaml
import json
import geopandas as gpd

gdf = gpd.read_file('output/residential/residential.csv')
gdf.to_file("output/residential/preview.geojson", driver='GeoJSON')

with open("output/residential/datapackage.json", 'r') as j, \
     open("output/residential/preview.geojson", 'r') as l, \
     open("output/residential/snapshot.json", 'w') as r:
    data = json.load(j)
    feed = json.load(l)
    data["resources"] = [{"name": "data-layer", "mediatype": "application/vnd.simplestyle-extended", "data": {"name": "data", "type": "FeatureCollection", "features": []}},{"name": "mapbox-background", "path": "mapbox://styles/gemeindescan/ck6rp249516tg1iqkmt48o4pz", "mediatype": "application/vnd.mapbox-vector-tile", "data":"" }]
    data['resources'][0]['data']['features'] = feed['features']
    json.dump(data, r)
```